In [1]:
from __future__ import annotations


import engine as e
import numpy as np
import pandas as pd
import tqdm as bar
import typing as t
import chess
from sklearn.model_selection import train_test_split

In [2]:
rng = np.random.default_rng()

def clamp(val, minval, maxval):
    return sorted((minval, val, maxval))[1]

class ValueOutOFRange(Exception):
    def __init__(self, axis: str) -> None:
        super().__init__(f'Values given for {axis}-axis are out of range.')



In [3]:
class Organism:
    def __init__(self, chrom: np.ndarray = None) -> None:
        self.chromosome = chrom
        self.fitness : float = 0.0 
    
    
    @classmethod
    def createRandomOrg(cls, chromLen: int = 1, 
            chromMin: float = np.finfo('float32').min, 
            chromMax: float = np.finfo('float32').max) -> Organism:

        chromosome : np.ndarray = rng.uniform(size = chromLen, low = chromMin, high = chromMax) 
        return cls(chromosome) 

    def mutate(self, mRate):
        self.chromosome += rng.normal(loc = 0, scale = mRate, size = self.chromosome.shape)
    
    @staticmethod
    def getChild(parent1 : Organism, parent2 : Organism, interpolFac : float) -> Organism:
        childChrom = parent1.chromosome * interpolFac + (1 - interpolFac) * parent2.chromosome 

        return Organism(childChrom)
    
    def __str__(self):
        return str(self.chromosome)


In [4]:
class Population:
    
    def __init__(self, 
            popSize: int = 100, 
            orgList: t.List = [],              
            fitness: Function2D = None 
            ) -> None:

        self.orgList: t.List = orgList 
        self.popSize: int = popSize
        self.fitFunc: Function2D = fitness
    
    @classmethod
    def initFromRandomOrgs(cls,
            popSize: int = 100, 
            orgData: t.Dict = {
                'len': 2, 
                'min': np.finfo('float32').min, 
                'max': np.finfo('float32').max
                }, 
            fitness: t.Callable = lambda x: np.max(x) 
            ) -> Population:

        orgList : t.List = [
                Organism.createRandomOrg(
                    chromLen = orgData['len'], 
                    chromMin = orgData['min'],
                    chromMax = orgData['max']) 
                for _ in range(popSize)] 

        return Population(popSize = popSize, 
                orgList = orgList,
                fitness = fitness)

   
    def truncationSelection(self, topCount: int) -> (t.List, t.List):
        n = len(self.orgList)
        for i, o in enumerate(self.orgList):
            #print(f'organism evaluated: {i}/{n}')
            
            try:
                o.fitness = self.fitFunc.evaluateSingle(o.chromosome)
            except ValueOutOFRange:
                o.fitness = np.finfo('float32').min
        
        s = sorted(self.orgList, key = lambda o : o.fitness, reverse = True)
        parents = s[:topCount] 

        return parents, s[(topCount+1):]

    def updateOrgList(self, parentsList: t.List, childrenList: t.List):
        self.orgList = parentsList + childrenList

    def getOrgs(self, *args) -> t.List:
        out = []
        for arg in args:
            out.append(self.orgList[arg])
           
        return out
    
    def __str__(self):
        _, top = self.truncationSelection(4)
        outs = ''
        
        for i in top:
            outs += str(i.chromosome) + '\n'
        
        return outs

In [5]:
class GeneticAlgorithm():  
    # kwarg: selectPer, interpolFac, mutationRate  
    def __init__(self, 
            population: Population = None, **kwargs
            ) -> None:

        selectPer: float   = kwargs.pop('selectPer', 0.20) 
        interpolFac: float = kwargs.pop('interpolFac', 0.5)
        mutationRate: float = kwargs.pop('mutationRate', 1.0)

        self.population: Population = population
        self.k:  int = int(population.popSize * clamp(selectPer, 0.0, 1.0))
        self.interpolFac: float = clamp(interpolFac, 0.0, 1.0)
        self.mutRate: float = mutationRate 

    @classmethod
    def initRandomPop(cls,
            popSize: int = 100, 
            orgData: t.Dict = {
                'len': 2, 
                'min': np.finfo('float32').min, 
                'max': np.finfo('float32').max
                }, 
            fitness: Function2D = None, 
            **kwargs,
            ) -> GeneticAlgorithm:

        p = Population.initFromRandomOrgs(popSize, orgData, fitness)  

        return cls(p, **kwargs)

    def selection(self) -> (t.List, t.List):
        # Using truncation selection 
        return self.population.truncationSelection(self.k) 

    def crossover(self, parentsList : t.List) -> t.List:
        numChildren = self.population.popSize - self.k 
        children : t.List = []
        high = len(parentsList) 
        for _ in range(0, numChildren):
            p : np.ndarray = rng.choice(a = high, size = 2) 

            p1 = parentsList[p[0]]
            p2 = parentsList[p[1]]

            children.append(Organism.getChild(p1, p2, self.interpolFac))

        return children

    def mutation(self, childrenList: t.List):
        for c in childrenList:
            c.mutate(self.mutRate)

    def updateOrgList(self, parentsList: t.List, childrenList: t.List):
        self.population.updateOrgList(parentsList, childrenList)

In [6]:
class EvaluationTraining:
    def __init__(self, depth, dspath):
        # dataset
        df = pd.read_csv(dspath)
        xtrain, xtest, ytrain, ytest = train_test_split(df['Fen'], df['San'], test_size = 0.33)
        
        self.xtrain = list(xtrain)
        self.xtest = list(xtest)
        self.ytrain = list(ytrain)
        self.ytest = list(ytest)
        
        # 
        self.engine = e.Engine(depth)
        
        
    def evaluateSingle(self, chrom):
        value = {
                chess.PAWN: chrom[0],
                chess.KNIGHT: chrom[1],
                chess.BISHOP: chrom[2],
                chess.ROOK: chrom[3],
                chess.QUEEN: chrom[4]
        }
        
        self.engine.setEvaluationParameters(value)
        fens = list(self.xtrain)
        out = self.engine.findmoves(fens)
        
        count = 0
        for i, o in enumerate(out):
            if self.ytrain[i] == o:
                count += 1
                
        return count / len(fens)
        

In [7]:
def GA():
    sf = EvaluationTraining(1, '../dataset.csv')

    ga = GeneticAlgorithm.initRandomPop(
            popSize = 20, 
            orgData = {'len': 5, 'min': 0, 'max': 100},
            fitness = sf,
            selectPer = 0.20,
            interpolFac = 0.5,
            mutationRate = 2.0
            )
    
    for g in range(0, 20):
        
        print(f'Generation = {g}')
        
        parents, rejected = ga.selection()
        children = ga.crossover(parents)
        ga.mutation(children)
        ga.updateOrgList(parents, children)
    
    print(str(ga.population.__str__()))
        

In [8]:
GA()

Generation = 0
Generation = 1
Generation = 2
Generation = 3
Generation = 4
Generation = 5
Generation = 6
Generation = 7
Generation = 8
Generation = 9
Generation = 10
Generation = 11
Generation = 12
Generation = 13
Generation = 14
Generation = 15
Generation = 16
Generation = 17
Generation = 18
Generation = 19
[53.27830595 73.97637989 43.20221017 22.77357944 48.86534008]
[47.39840231 74.38169178 43.41427222 28.36416514 49.07897037]
[17.25874132 49.02260222 43.3312498  23.84124125 18.71424985]
[49.67033583 40.37454142 15.22984286 38.41209251 46.72360236]
[50.08089067 74.12383888 40.82985635 27.85113907 49.58274745]
[98.31018456 49.9156042  16.9012708  41.49663811 79.19989251]
[ 6.48541929 62.87806136 43.17391733 21.71553107 27.38929793]
[14.30234211 52.55208873 46.43910487 24.08644928 17.39000931]
[20.58314631  0.15860612 20.07413873 35.63013456  9.07546971]
[ 8.06143393 11.43870552 16.89794193 37.96963908 15.55664101]
[13.13389403 53.51972033 49.13894935 27.55887337 13.85360848]
[62.9237